In [1]:
import pyspark
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from settings import *
import math

# Read data, Split to train test

In [2]:
# Sorting data by time
rdd =  sc.textFile(RATINGS_10M).map(lambda line: [float(x) for x in line.split('::')]).sortBy(lambda x: x[3],False)
size = rdd.count()

In [3]:
# Processing data to structure: Rating(user=62510, product=34148, rating=3.0)
train = rdd.zipWithIndex().filter(lambda x: x[-1] < size*0.6).map(lambda x: Rating(int(x[0][0]), int(x[0][1]), x[0][2]))
testdata = rdd.zipWithIndex().filter(lambda x: x[-1] > size*0.6).map(lambda x: Rating(int(x[0][0]), int(x[0][1]), x[0][2]))

# Compute user bias

In [4]:
score_mean = train.map(lambda data: data[2]).mean()
user_bias = train.map(lambda data: (data[0], data[2] - score_mean)).groupByKey().map(lambda data: (data[0], sum(data[1])/len(data[1]))).collectAsMap()

# Compute movie bias

In [5]:
movie_bias = train.map(lambda data: (data[1], data[2] - score_mean - user_bias[data[0]])).groupByKey().map(lambda data: (data[0], sum(data[1])/len(data[1]))).collectAsMap()

In [6]:
train_rm_user = train.map(lambda rating: Rating(rating.user, rating.product, rating.rating - user_bias[rating.user]))
train_rm_movie = train.map(lambda rating: Rating(rating.user, rating.product, rating.rating - user_bias[rating.user] - movie_bias[rating.product]))


In [7]:
#print movie_count
#print movie_score_bias_sum
#print movie_bias
print train.first()
print train_rm_user.first()
print train_rm_movie.first()

Rating(user=62510, product=34148, rating=3.0)
Rating(user=62510, product=34148, rating=2.5208408027796696)
Rating(user=62510, product=34148, rating=2.289214762263664)


# Model Evaluation - removed user bias

In [8]:
test = testdata.map(lambda p: (p[0], p[1]))

In [9]:
# Training ALS Model
latentFactors = 10
numIterations = 25
regParameter = 0.1

model = ALS.train(train_rm_user, latentFactors, numIterations, regParameter)

In [10]:
predictions = model.predictAll(test).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = testdata.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)

In [11]:
RMSE = ratesAndPreds.map(lambda r: math.sqrt((r[1][0] - r[1][1])**2)).mean()
print("Root Mean Squared Error = " + str(RMSE))

Root Mean Squared Error = 0.78307085766


# Recommendation System

In [8]:
# Trying recommendation System 

testUsers = train_rm_user.first()[0]
print testUsers

62510


In [9]:
traindict = train.map(lambda data: (data[0], [data[1]])).reduceByKey(lambda a,b: a+b).collectAsMap()

In [10]:
# Creat recommended movies that user have not watched before
topMovies = model.recommendProducts(testUsers, 100)
filteredMovies = []
for row in topMovies:
    if not row[1] in traindict[row[0]]:
        filteredMovies.append(row)


In [11]:
# Print out filtered Movies as string (names)
movies10M_dict =  sc.textFile(MOVIES_10M).map(lambda line: line.split('::')).map(lambda x: (float(x[0]), x[1])).collectAsMap()

for i in range(5):
    print movies10M_dict[filteredMovies[i][1]]

Shadows of Forgotten Ancestors (1964)
Satan's Tango (Sátántangó) (1994)
More (1998)
Sun Shines Bright, The (1953)
Man Who Planted Trees, The (Homme qui plantait des arbres, L') (1987)
